In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [20]:
df_train = pd.read_csv('train.csv', index_col='id')
df_test = pd.read_csv('test.csv', index_col='id')

In [8]:
import warnings

warnings.filterwarnings('ignore')

In [10]:
df_train.head()

,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,...,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
id,,,,,,,,,,,,,,,,,,,,,
0,e,8.80,f,s,u,f,a,c,w,4.51,...,NaN,NaN,w,NaN,NaN,f,f,NaN,d,a
1,p,4.51,x,h,o,f,a,c,n,4.79,...,NaN,y,o,NaN,NaN,t,z,NaN,d,w
2,e,6.94,f,s,b,f,x,c,w,6.85,...,NaN,s,n,NaN,NaN,f,f,NaN,l,w
3,e,3.88,f,y,g,f,s,NaN,g,4.16,...,NaN,NaN,w,NaN,NaN,f,f,NaN,d,u
4,e,5.85,x,l,w,f,d,NaN,w,3.37,...,NaN,NaN,w,NaN,NaN,f,f,NaN,g,a


In [11]:
df_train.isnull().sum()

class                         0
cap-diameter                  4
cap-shape                    40
cap-surface              671023
cap-color                    12
does-bruise-or-bleed          8
gill-attachment          523936
gill-spacing            1258435
gill-color                   57
stem-height                   0
stem-width                    0
stem-root               2757023
stem-surface            1980861
stem-color                   38
veil-type               2957493
veil-color              2740947
has-ring                     24
ring-type                128880
spore-print-color       2849682
habitat                      45
season                        0
dtype: int64

In [12]:
X_train = df_train.drop(df_train.columns[0], axis=1)
Y_train = df_train[df_train.columns[0]]
X_test = df_test.drop(df_test.columns[0], axis=1)
Y_test = df_test[df_test.columns[0]]


In [13]:
Y_train[Y_train == 'e'] = 0

Y_train[Y_train == 'p'] = 1

Y_test[Y_test == 'e'] = 0

Y_test[Y_test == 'p'] = 1


In [2]:
#hotencoding

import csv
"""Data colums that need hot encoding
id,class,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-root,stem-surface,
stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
"""
train_DataFrame = pd.read_csv('train.csv', index_col= 'id')
train_DataFrame.columns
#these should be all the categorical columns that need to be encoded, add any additional ones I missed if you find any
encode_columns = train_DataFrame[['class', 'cap-shape', 'cap-surface', 'cap-color', 'does-bruise-or-bleed', 'gill-attachment',
                                  'gill-spacing','gill-color','stem-root','stem-surface', 'stem-color','veil-type', 
                                  'veil-color','has-ring','ring-type','spore-print-color','habitat','season']]


#rest of code in next code cell


In [26]:
from sklearn.preprocessing import MultiLabelBinarizer

#dictionary has column key mapped to a dictionary that contains a "classes" key mapped to the different classes in the respective column
#and it has and "encoded" key mapped to the encoding of the column 
encoded_Cols = {}
print(encode_columns.columns.tolist())
for column in encode_columns.columns.tolist():
    print(column)
    mlb = MultiLabelBinarizer()
    encode = mlb.fit_transform(encode_columns[column])
    encoded_Cols[column] = {'classes': mlb.classes_,
                              'encoded' : encoded}
    
#I think this errors for the reason I mentioned in the next code block below





['class', 'cap-shape', 'cap-surface', 'cap-color', 'does-bruise-or-bleed', 'gill-attachment', 'gill-spacing', 'gill-color', 'stem-root', 'stem-surface', 'stem-color', 'veil-type', 'veil-color', 'has-ring', 'ring-type', 'spore-print-color', 'habitat', 'season']
class
cap-shape


TypeError: 'float' object is not iterable

In [25]:
#example of how it should work.
mlbs = MultiLabelBinarizer()
encoded = mlbs.fit_transform(encode_columns['class'])
print(encoded)
print(mlbs.classes_)

#print unique values in the column. Not all of it is categorical and has column names like ring-type (859165).
#maybe my csv file's data got screwed up somehow idk.
#however on the kaggle it seems that the cap-shape column should only have values x, f, x, b, and o based on the drop down menu on the cap-shape
#column in train.csv

print(set(encode_columns["cap-shape"]))
print(set(df_train["cap-shape"]))

[[1 0]
 [0 1]
 [1 0]
 ...
 [0 1]
 [1 0]
 [0 1]]
['e' 'p']
{'c', '24.16', 'm', 'u', 't', 'b', '7 x', 'p', '8', '2.94', '6.44', '2.82', '7.6', '49.21', 'i', '3.55', '8.3', '6.53', 'd', '0.82', '3.6', '2.77', 'a', '12.62', '19.06', nan, '10.46', '10.13', 'b f', '6.21', '54.78', 's', '4.3', 'k', '3 x', 'y', 'f', 'x', '3.37', '3.25', '4.22', '11.12', '3.04', '2.85', 'is p', 'l', 'ring-type', 'g', 'h', '7.21', '17.44', '3.91', '7.43', '21.56', 'z', '3.52', '20.25', '4.33', '5 f', '8.29', 'is s', '9.13', '1.66', 'w', '2.63', '7.41', 'e', '6 x', '19.04', '5.15', 'n', '4.97', '19.29', 'r', 'o'}
{'c', '24.16', 'm', 'u', 't', 'b', '7 x', 'p', '8', '2.94', '6.44', '2.82', '7.6', '49.21', 'i', '3.55', '8.3', '6.53', 'd', '0.82', '3.6', '2.77', 'a', '12.62', '19.06', nan, '10.46', '10.13', 'b f', '6.21', '54.78', 's', '4.3', 'k', '3 x', 'y', 'f', 'x', '3.37', '3.25', '4.22', '11.12', '3.04', '2.85', 'is p', 'l', 'ring-type', 'g', 'h', '7.21', '17.44', '3.91', '7.43', '21.56', 'z', '3.52', '20.25', '

In [ ]:
def train_using_entropy(X, y):
    # Decision tree with entropy
    dft_entropy = DecisionTreeClassifier(
        criterion="entropy", random_state=100,
        max_depth=3, min_samples_leaf=5)

    # Performing training
    dft_entropy.fit(X, y)
    return dft_entropy

In [ ]:
y_pred = train_using_entropy(X_train, Y_train).predict(X_test)
print("Accuracy : ", accuracy_score(Y_test, y_pred)*100)

In [ ]:
import xgboost as xgb
# define data_dmatrix
#https://gist.github.com/pb111/cc341409081dffa5e9eaf60d79562a03
data_dmatrix = xgb.DMatrix(data=X_train,label=Y_train)

In [ ]:
# import XGBClassifier
from xgboost import XGBClassifier


# declare parameters
params = {
            'objective':'binary:logistic',
            'max_depth': 4,
            'alpha': 10,
            'learning_rate': 1.0,
            'n_estimators':100
        }
            
            
            
# instantiate the classifier 
xgb_clf = XGBClassifier(**params)



# fit the classifier to the training data
xgb_clf.fit(X_train, Y_train)

In [ ]:
# make predictions on test data
y_pred2 = xgb_clf.predict(X_test)

In [ ]:

print('XGBoost model accuracy score: {0:0.4f}'. format(accuracy_score(Y_test, y_pred)))